# Parte 1 - Interpolação

## 1.1 Pacotes e funções necessários

In [ ]:
from scipy.interpolate import interp1d  # funções lineares e splines
from scipy.interpolate.rbf import Rbf  # funções de base radial

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## 1.2 Obtendo e estudando os dados

In [ ]:
"""
Vamos baixar a tabela .csv na qual trabalharemos.
Endereço dos dados: http://goo.gl/h94Mqk
"""

data_petro = np.genfromtxt('table.csv', delimiter=',', dtype=None)

In [ ]:
# Vendo o tamanho dos dados.
print('A tabela tem {0} linhas e {1} colunas.'.format(np.shape(data_petro)[0],
                                                       np.shape(data_petro)[1]))

In [ ]:
# Vendo o cabeçalho.
data_petro[0:4,0:]

In [ ]:
"""
Tanto a primeira linha quanto a primeira coluna representam os rótulos dos dados.
Para obter os dados, começamos da segunda linha/coluna:
"""

data_petro = data_petro[1:,1:]

In [ ]:
data_petro

In [ ]:
"""
Dados decrescentes! Vamos invertê-los.
Usamos np.flipud(data_petro) ou data_petro[::-1].
"""

data_petro = np.flipud(data_petro)
data_petro

In [ ]:
"""
Usaremos a coluna 'Adj Close', o preço de fechamento ajustado.
"""

data_petro[:, 5:]

In [ ]:
# Separando 'Adj Close' e visualizando os dados:
adj_close = np.array(data_petro[:, 5:].flatten(), dtype=float)  # "Achatar" o vetor para 1 dimensão
plt.figure(figsize=(16, 8))  # aumentando o tamanho da figura.
plt.plot(adj_close, 'ko')
plt.xlabel('Dias')
plt.ylabel('Preço de fechamento ajustado')

## 1.3 Interpolando: função scipy.interpolate.interp1d

In [ ]:
# Determinando o tamanho do vetor adj_close.
fim = np.shape(adj_close)[0]
fim

In [ ]:
"""
A função interp1d gera uma função.
O argumento 'kind' especifica o tipo de interpolação utilizada. O padrão é 'linear'.
* Argumentos válidos: 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic'. 
* Os argumentos 'slinear', 'quadratic' e 'cubic' se referem à interpolação com uma spline
de primeira, segunda ou terceira ordem.
* Caso seja utilizado um inteiro, ele expressará a ordem da spline que será utilizada.

* Para mais informações, veja a documentação do SciPy:
http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.interpolate.interp1d.html
"""

adj_x = np.linspace(0, fim, fim, endpoint=True)  # criando eixo X com dias.
interp_adj = interp1d(adj_x, adj_close)

In [ ]:
intp_adjclose = interp_adj(adj_x)
plt.figure(figsize=(16, 8))  # aumentando o tamanho da figura.
plot1 = plt.plot(adj_x, adj_close, 'ko', label='orig')
plot1 = plt.plot(adj_x, intp_adjclose, '--', label='intp')

leg = plt.legend()

In [ ]:
# Tomando os primeiros 10 dias. Criando também um eixo X
adj_peda = adj_close[:10]
eixo_x = np.linspace(0, 10)

# Interpolando o resultado entre os 10 dias.
intp_peda = interp_adj(eixo_x)

# Plotando o resultado.
plt.figure(figsize=(16, 8))
plot2 = plt.plot(adj_peda, 'ko', markersize=12, label='orig')  # os pontos originais.
plot2 = plt.plot(eixo_x, intp_peda, '-', label='lin')

# Testando também outras formas...
# kind='nearest':
near_adj = interp1d(adj_x, adj_close, kind='nearest')
near_peda = near_adj(eixo_x)
plot2 = plt.plot(eixo_x, near_peda, '-', label='near')

"""
Para testar em casa.
kind='cubic':
"""

# cub_adj = interp1d(adj_x, adj_close, kind='cubic')
# cub_peda = cub_adj(eixo_x)
# plot2 = plt.plot(eixo_x, cub_peda, '-', label='cub')

leg = plt.legend()

## 1.4 Interpolando: função scipy.interpolate.Rbf

In [ ]:
"""
Da mesma forma que interp1d, a função Rbf gera uma função.
O argumento 'function' especifica o tipo de interpolação utilizada. O padrão é 'multiquadric'.
* Argumentos válidos: 'multiquadric', 'inverse', 'gaussian', 'linear', 'cubic', 'quintic', 'thin_plate'. 
* As funções-base são:
 - 'multiquadric': sqrt((r/self.epsilon)**2 + 1)
 - 'inverse': 1.0/sqrt((r/self.epsilon)**2 + 1)
 - 'gaussian': exp(-(r/self.epsilon)**2)
 - 'linear': r
 - 'cubic': r**3
 - 'quintic': r**5
 - 'thin_plate': r**2 * log(r)

* Para mais informações, veja a documentação do SciPy:
http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.interpolate.Rbf.html
"""

adj_x = np.linspace(0, fim, fim, endpoint=True)  # Criando eixo X com dias.
rbf_adj = Rbf(adj_x, adj_close)

In [ ]:
# Tomando os primeiros 60 dias. Criando também um eixo X
adj_bimes = adj_close[:60]  
eixo_x = np.linspace(0, 60, 500)

# Interpolando o resultado entre os 10 dias.
plt.figure(figsize=(16, 8))

# Plotando os pontos originais.
plot3 = plt.plot(adj_bimes, 'ko', markersize=12)  # os pontos originais.

# Interpolando.
# function = 'linear':
rbf_adj = Rbf(adj_x, adj_close, function='linear')
lin_bimes = rbf_adj(eixo_x)
plot3 = plt.plot(eixo_x, lin_bimes, '-', label='lin')

# function = 'cubic':
rbf_adj = Rbf(adj_x, adj_close, function='cubic')
cub_bimes = rbf_adj(eixo_x)
plot3 = plt.plot(eixo_x, cub_bimes, '-', label='cubic')

"""
Para testar em casa.
function = 'gaussian':
"""

#rbf_adj = Rbf(adj_x, adj_close, function='gaussian')
#gauss_bimes = rbf_adj(eixo_x)
#plot3 = plt.plot(eixo_x, gauss_bimes, 'o', label='gauss')

leg = plt.legend()

# Fim da Parte 1.